### Norms and cosine distance
1. [Wolfram](https://mathworld.wolfram.com/MatrixNorm.html)
2. [CS Cornell](https://www.cs.cornell.edu/courses/cs4220/2013sp/CVLBook/chap5.pdf)
3. [MATLAB](https://la.mathworks.com/help/matlab/ref/norm.html)
4. [Numpy](https://numpy.org/doc/stable/reference/generated/numpy.linalg.norm.html)
5. [Cosine distance](https://en.wikipedia.org/wiki/Cosine_similarity). It is only defined for vectors? $\lang x, y \rang = \|x\| \|y\| \cos\theta$ is true for any inner product? An example of matrix inner product?<br>
    - [Matrix inner product](https://sharmaeklavya2.github.io/theoremdep/nodes/linear-algebra/matrices/inner-product-space.html)

In [22]:
import numpy as np
import timeit
# import inspect
# print(inspect.getsource(np.linalg.norm))

def my_norm_switch(   
        A: np.ndarray,
        type = 'fro') -> float:
    # Parameters pre-processing.
    type = str(type).lower()

    my_switch = {
    'fro': np.sqrt(np.sum(np.power(A, 2))),
    # By the min-max theorem, the 2-norm matrix is the largest singular value.
    '2': np.linalg.svd(A)[1][0],
    '1': np.max(np.sum(A, axis=0)),
    'inf' : np.max(np.sum(A, axis=1))
    }

    return my_switch.get(type, 'Invalid type')

def my_norm_if(   
        A: np.ndarray,
        type = 'fro') -> float:
    if A.ndim == 2:
        if type=='fro':
            ret = np.sqrt(np.sum(np.power(A, 2)))
            # ret = np.sqrt(np.trace(A @ A.T))
        elif type == 2:
            # By the min-max theorem, the 2-norm matrix is the largest singular value.
            ret =  np.linalg.svd(A)[1][0]
        elif type == 1:
            ret = np.max(np.sum(A, axis=0))
        elif type == np.inf:
            ret = np.max(np.sum(A, axis=1))
        else:
            raise ValueError("Invalid norm type for matrices.")
    else:
        raise ValueError("Invalid matrix dimension (must be 2D).")
    
    return ret

def matrixInnerProduct(A, B):
    # The conjugate is the generalization (complex numbers) of the inner product to matrices.
    return np.trace(A @ np.matrix.conjugate(B.T))

def cosineDistance(a, b):
    if a.ndim == 1 and b.ndim == 1:
        Sc = np.dot(a, b) / (np.linalg.norm(a, ord=2) * np.linalg.norm(b, ord=2))
    elif a.ndim == 2 and b.ndim == 2:
        Sc = matrixInnerProduct(a, b) / np.sqrt(matrixInnerProduct(a, a) * matrixInnerProduct(b, b))
    else:
        raise ValueError("Cosine distance is only defined for vectors and matrices.")
    return 1 - Sc

### Examples

In [42]:
# Validate the cosine distance for vectors.
a = np.array([1,0,1])
b = np.array([1,0,0])
c = np.array([1,1,1])

assert cosineDistance(a, b) >= 0
assert cosineDistance(a, b) == cosineDistance(b, a)
assert cosineDistance(a, b) <= cosineDistance(a, c) + cosineDistance(c, b)


In [41]:
# Validate the cosine distance for matrices.
A = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
B = np.array([[1, 2, 3], [4, 0, 6], [7, 8, 9]])
# It is not defined for zeros matrices.
# B = np.zeros(A.shape)
C = np.ones(A.shape)

assert cosineDistance(A, B) >= 0
assert cosineDistance(A, B) == cosineDistance(B, A)
assert cosineDistance(A, B) <= cosineDistance(A, C) + cosineDistance(C, B)

In [40]:
A = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
B = np.zeros(A.shape)
# It is not defined for zeros matrices.
cosineDistance(A, B)

/var/folders/81/k3c_c4qs1ldcb8jbks6zcshw0000gn/T/ipykernel_72206/845792885.py:53: RuntimeWarning: invalid value encountered in double_scalars
  Sc = matrixInnerProduct(A, B) / np.sqrt(matrixInnerProduct(A, A) * matrixInnerProduct(B,B))


nan

In [190]:
# Test if our norms are identical to numpy's.
As = [np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]]), np.random.rand(3, 3)]
types = [np.inf, 'fro', 2, 1]

for A in As:
    for type in types:
        assert (np.linalg.norm(A, ord=type) - my_norm_if(A, type=type)) < 1e-10
        assert (my_norm_switch(A, type=type) - my_norm_if(A, type=type)) < 1e-10

### Time complexity

In [189]:
print("Numpy execution:", timeit.timeit(\
    lambda: np.linalg.norm(A, ord=1), number=10000), "seconds")

print("If execution:", timeit.timeit(\
    lambda: my_norm_if(A, 1), number=10000), "seconds")

print("Switch execution:", timeit.timeit(\
    lambda: my_norm_switch(A, '1'), number=10000), "seconds")

Numpy execution: 0.042156916999374516 seconds
If execution: 0.0428554999998596 seconds
Switch execution: 0.19274270799996884 seconds
